# <2팀 크롤링 과제>**

## Q63. 네이버 검색에서 관심 주제와 기사 수를 입력하면 그만큼 출력하는 크롤링 프로그램을 작성하여 다음 사항을 수행하세요.
기사 제목과 링크를 데이터프레임으로 가져온 후 본문 내용에서 한글 단어만 출력하여 형태소 분석 최근 국내 'AI'와 관련하여 가장 많이 언급되는 단어에 대한 빈도분석 및 시각화 AI 분야에서의 유의할 만한 트랜드 정보 및 인사이트 도출

### 1. 검색 키워드, 기사 수 입력 프로그램 생성

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [18]:
from bs4 import BeautifulSoup 
import urllib
import urllib.request as rq
import pandas as pd


global news_df

def News():
    global keyword, num, news_df, url,page_num
    
    keyword = input('검색 키워드를 입력해주세요')
    keyword = urllib.parse.quote(keyword) #퍼센트인코딩 #URL에 문자를 표현하는 인코딩방법(URL에 한글이 섞이면 오류 발생)
    num = int(input('필요한 뉴스기사의 숫자를 입력해주세요'))
    
    news_df = pd.DataFrame(columns=['title','url']) 
    
    page_num=1
    i=0 #크롤링한 기사의 수
    while num > i:
        url = 'https://search.naver.com/search.naver?where=news&sm=tab_jum&query='+keyword+'&start='+str(page_num)
        html = rq.urlopen(url)
        bs = BeautifulSoup(html,'html.parser')
        texts = bs.find_all(class_='news_tit')
        links = bs.select('div.news_wrap.api_ani_send > div > div.news_info > div.info_group > a:nth-child(3)') # 네이버 링크 추출

        for text in texts:
            news_df.loc[i,'title'] = text.get_text()
        
        for link in links:
            news_df.loc[i, 'url'] = link['href']
            i +=1
            if i == num:
                break
        page_num +=10
    return news_df

News()

#AI, 인공지능

검색 키워드를 입력해주세요ai
필요한 뉴스기사의 숫자를 입력해주세요200


,title,url
0,[경남소식] 도 교육청 'AI시대 대비' 수학 교사 연수,https://n.news.naver.com/mnews/article/014/000...
1,NaN,https://n.news.naver.com/mnews/article/057/000...
2,NaN,https://n.news.naver.com/mnews/article/011/000...
3,NaN,https://n.news.naver.com/mnews/article/001/001...
4,NaN,https://n.news.naver.com/mnews/article/092/000...
...,...,...
195,NaN,https://n.news.naver.com/mnews/article/658/000...
196,NaN,https://n.news.naver.com/mnews/article/011/000...
197,NaN,https://n.news.naver.com/mnews/article/003/001...
198,"이글루, KISA 사이버보안 AI 데이터셋 구축 사업 수주",https://n.news.naver.com/mnews/article/008/000...


- 중복값 제거

In [3]:
news_df[news_df['title'].duplicated() == True] 
news_df=news_df.drop_duplicates(subset=['title'])
news_df.describe()

,title,url
count,1613,1613
unique,1613,1613
top,AI로 신소재 개발 속도 높인다,http://www.fnnews.com/news/202208091117408193
freq,1,1


In [4]:
news_df[news_df['title'].duplicated() == True] #688중복

,title,url


In [5]:
# 중복값제거
news_df

,title,url
0,AI로 신소재 개발 속도 높인다,http://www.fnnews.com/news/202208091117408193
1,"""AI가 예측한 로또번호입니다""…6만 명이 600억 원 뜯겼다",http://www.mbn.co.kr/pages/news/newsView.php?n...
2,잡코리아 ‘AI 추천&검색 서비스’ 론칭,https://www.sedaily.com/NewsView/269QB4Z3A3
3,"카카오벤처스 ""AI 임상시험 분석기업 액트노바에 시드투자""",https://www.yna.co.kr/view/AKR2022080903130001...
4,"비투엔, 광주에 AI융합센터 개소···내년 디지털헬스 서비스 선보여",https://zdnet.co.kr/view/?no=20220809172036
...,...,...
1995,음식물처리기에도 AI 신기술 도입…웰릭스 팅크웨어 에콥 등 줄줄이 출시,http://news.mt.co.kr/mtview.php?no=20220727142...
1996,"순천시, AI 반려로봇 '루미' 어르신 돌봄 시작",http://daily.hankooki.com/news/articleView.htm...
1997,"""축산업 AI 기술 시장 선두 주자로""…인트플로우, 축산 AI 학습용 데이터 구축...",http://www.aitimes.com/news/articleView.html?i...
1998,"[산업 이모저모] SKT, 화신과 '웰딩 AI 솔루션' 계약 체결...'인더스트리얼...",http://www.greened.kr/news/articleView.html?id...


### 2. 본문 긁어오기

In [20]:
import requests as req
# 본문읽어오기
headers = {"User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/96.0.4664.45 Safari/537.36"}

li = []
for i in news_df.url:
    url = i
    html = req.get(url, headers = headers)
    bs = BeautifulSoup(html.text,'lxml')
    texts = bs.find_all(class_ = "_article_content")
    for text in texts:
        t = text.get_text()
        li.append(t)
li

['\nUNIST 김재업 교수팀 \'AI 고분자 시뮬레이션 기술\' 개발기존 딥러닝기법 활용한 것보다 6배 이상 빠른 결론 얻어딥러닝 위한 데이터 준비·훈련 시간 포함해도 4배 빨라져 \n\n\n\n인공지능. 게티이미지 제공  [파이낸셜뉴스] 울산과학기술원(UNIST) 물리학과 김재업 교수팀은 고분자 신소재를 빠르게 개발할 수 있는 \'인공지능(AI) 고분자 시뮬레이션 기술\'을 개발했다. 연구진이 개발한 기술은 기존 딥러닝 기법을 활용한 시뮬레이션보다 6배 이상 빠르게 결론을 얻어낼 수 있다.   김재업 교수는 9일 "이 기술은 AI 딥러닝 위한 데이터 준비와 훈련에 드는 시간 포함해도 기존 대비 최소 4배 이상 빠르다"고 설명했다. 또한 제1저자인 고등과학원(KIAS) 용대성 연구원(UNIST 박사 졸업)은 "지금까지는 작은 고분자계에서 매우 제한적으로 활용됐던 \'장이론 시뮬레이션\'을 대면적 박막이나 복잡한 형상이 예상되는 고분자에도 사용할 길이 열렸다"고 말했다.   고분자는 옷감이나 플라스틱의 재료로 널리 쓰이고 있다. 사용목적에 따라 소재의 분자를 설계하는데 시뮬레이션을 활용한다.   최근에 새롭게 개발된 \'랑주뱅 장이론 시뮬레이션(L-FTS)\'도 최적의 고분자 구조를 찾는 작업을 수십만번 이상 반복해야 한다. 계산량이 너무 많아 고성능 그래픽 처리장치(GPU)를 사용해도 시뮬레이션 한 번에 며칠씩 걸렸다.   또 기존에 사용하는 반복법은 대략적인 예측갑에서 실제 결과까지 거리를 계산해 예측치를 수정하는 과정을 반복해 이 지점을 찾아냈는데, 한 번 찾을 때마다 예측 작업을 50회 정도 반복해야 했다.   연구진은 딥러닝을 이용해 반복법의 단점을 해결했다. 인공신경망에 많은 데이터를 주고 훈련을 진행해 예측치를 더 정확하게 얻었다. 이 기술을 사용해 50회씩 반복하던 예측을 2~4회로 줄일 수 있었다. 그결과 기존보다 6배 이상 빠르게 시뮬레이션을 수행할 수 있다. 인공신경망 훈련을 위한 데이터 준비와 훈련에 드는 시간을 포함해도 기존 대비 최소

In [21]:
# 문자 정리
import re
text = ' '.join(li) #list들을 공백으로 구분해서 연결
text1 = re.sub('[^가-힣]+',' ',text) #한글 이외는 공백으로 대체
text1

' 김재업 교수팀 고분자 시뮬레이션 기술 개발기존 딥러닝기법 활용한 것보다 배 이상 빠른 결론 얻어딥러닝 위한 데이터 준비 훈련 시간 포함해도 배 빨라져 인공지능 게티이미지 제공 파이낸셜뉴스 울산과학기술원 물리학과 김재업 교수팀은 고분자 신소재를 빠르게 개발할 수 있는 인공지능 고분자 시뮬레이션 기술 을 개발했다 연구진이 개발한 기술은 기존 딥러닝 기법을 활용한 시뮬레이션보다 배 이상 빠르게 결론을 얻어낼 수 있다 김재업 교수는 일 이 기술은 딥러닝 위한 데이터 준비와 훈련에 드는 시간 포함해도 기존 대비 최소 배 이상 빠르다 고 설명했다 또한 제 저자인 고등과학원 용대성 연구원 박사 졸업 은 지금까지는 작은 고분자계에서 매우 제한적으로 활용됐던 장이론 시뮬레이션 을 대면적 박막이나 복잡한 형상이 예상되는 고분자에도 사용할 길이 열렸다 고 말했다 고분자는 옷감이나 플라스틱의 재료로 널리 쓰이고 있다 사용목적에 따라 소재의 분자를 설계하는데 시뮬레이션을 활용한다 최근에 새롭게 개발된 랑주뱅 장이론 시뮬레이션 도 최적의 고분자 구조를 찾는 작업을 수십만번 이상 반복해야 한다 계산량이 너무 많아 고성능 그래픽 처리장치 를 사용해도 시뮬레이션 한 번에 며칠씩 걸렸다 또 기존에 사용하는 반복법은 대략적인 예측갑에서 실제 결과까지 거리를 계산해 예측치를 수정하는 과정을 반복해 이 지점을 찾아냈는데 한 번 찾을 때마다 예측 작업을 회 정도 반복해야 했다 연구진은 딥러닝을 이용해 반복법의 단점을 해결했다 인공신경망에 많은 데이터를 주고 훈련을 진행해 예측치를 더 정확하게 얻었다 이 기술을 사용해 회씩 반복하던 예측을 회로 줄일 수 있었다 그결과 기존보다 배 이상 빠르게 시뮬레이션을 수행할 수 있다 인공신경망 훈련을 위한 데이터 준비와 훈련에 드는 시간을 포함해도 기존 대비 최소 배 이상 속도가 향상됐다 김재업 교수는 이 기술은 심층인공신경망이 예측한 답을 그대로 사용하는 게 아니라 예측치와 정답의 차이를 다시 계산해 새로운 입력값을 부여해 더 정밀한 예측이 가능하다 고 설명

In [22]:
# 형태소 분석
#konlpy(콘엘피) = 불용어 처리 전
from konlpy.tag import Okt
from collections import Counter
okt = Okt() #객체형성

morph = okt.pos(text1) #형태소로 변환 #word= 단어 / tag=품사
morph

noun_list=[]
for word, tag in morph: #word= 단어 / tag=품사
    if(tag=='Noun') & (len(word)>1): #noun이고 단어가 1글자 이상인것(1글자는 보통 조사이기때문)
#         if word not in stop_word:
        noun_list.append(word)

count = Counter(noun_list) #단어별 숫자세주기

# 2022
data_bf =dict(count.most_common(30)) #빈도수 정리 내림차
series = pd.Series(data_bf)
df1_bf=pd.DataFrame(series, columns=['cnt'])
df1_bf

,cnt
지능,488
기술,478
인공,465
활용,366
데이터,361
금융,358
분야,344
개발,312
서비스,270
제공,265


In [23]:
# 불용어 처리
#konlpy(콘엘피) = 형태소 분석
from konlpy.tag import Okt
from collections import Counter
okt = Okt() #객체형성

morph = okt.pos(text) #형태소로 변환 #word= 단어 / tag=품사
morph

noun_list =[]
stop_word = ['AI', '인공', '지능', '기술', '서울', '개발', '활용', '제공', '통해', '데이터', '한국', '회사', '제공', '단계', '서비스']
for word, tag in morph: #word= 단어 / tag=품사
    if(tag=='Noun') & (len(word)>1): #noun이고 단어가 1글자 이상인것(1글자는 보통 조사이기때문)
        if word not in stop_word:
            noun_list.append(word)

count = Counter(noun_list) #단어별 숫자세주기

# 2022
data_af =dict(count.most_common(30)) #빈도수 정리 내림차
series = pd.Series(data_af)
df_af=pd.DataFrame(series, columns=['cnt'])
df_af

,cnt
금융,358
분야,344
교육,242
기업,205
의료,180
구축,173
산업,171
기반,169
연구,162
사업,160


### 3. wordcloud

In [24]:
from wordcloud import WordCloud
import matplotlib.pyplot as plt
import seaborn as sns
plt.figure(figsize=(10,8))

plt.subplot(1,2,1)
wordcloud = WordCloud(font_path='../visualization/dataset/malgun.ttf', background_color='white',
                     colormap='flare',width=1500,height=1000).generate_from_frequencies(data_bf)
plt.imshow(wordcloud)
plt.axis('off')

plt.subplot(1,2,2)
wordcloud = WordCloud(font_path='../visualization/dataset/malgun.ttf', background_color='white',
                     colormap='flare',width=1500,height=1000).generate_from_frequencies(data_af)
plt.imshow(wordcloud)
plt.axis('off')

ModuleNotFoundError: No module named 'wordcloud'

### 4. 트렌드 분석 및 인사이트

- AI관련 핵심 키워드
> 기술, 사업, 데이터, 솔루션
- 트렌드 및 인사이트 도출
> 교육, 인재, 양성 : AI 교육이 활성화되고있음 <br>
> 병원, 의료 : 의료분야의 AI 연구 활발 <br>
> 삼성, 반도체 : AI반도체가 국가적 전략기술로 부각되는 가운데, 삼성이 AI인재 양성계획 밝힘 <br>
> 금융, 하나은행,증권 : 금융권의 디지털 전환이 가속화. AI 기반 금융 서비스 구축. <br>
> 카카오, 상담: 카카오는 현재 AI분야에서 활발한 사업확장 중. AI상담챗봇, 카카오브레인의 AI작가, 기술윤리위원회(AI윤리규정 준수 여부)등을 신설.